#  Model Fitness

Primero cargo el dataset y le doy un primer vistazo

In [ ]:
# Importamos las librerías necesarias
import pandas as pd

# Cargamos el dataset
df = pd.read_csv('/datasets/gym_churn_us.csv')

# Mostramos las primeras filas del dataset para familiarizarnos con los datos
df.head()


In [ ]:
# Observamos la estructura del DataFrame
df.info()


🧱 Revisión estructural del dataset El dataset contiene 4,000 filas y 14 columnas.

No hay valores nulos en ninguna de las columnas.

La mayoría de las columnas son de tipo int64, lo cual indica que son variables numéricas enteras.

Cuatro columnas son de tipo float64, lo cual tiene sentido ya que representan medidas continuas, como la frecuencia de clases o los gastos adicionales.

In [ ]:
# Estadísticas descriptivas de todas las columnas numéricas
df.describe()

Edad (Age): El rango va de 18 a 41 años, con una media cercana a los 29 años. La mayoría de los clientes están en el rango de edad joven-adulto.

Lifetime: Algunos clientes han estado menos de un mes (0 meses) en el gimnasio, mientras que otros llevan hasta 31 meses. La mediana es de 3 meses, lo cual sugiere que muchos clientes son relativamente nuevos.

Contract_period: Tiene una media de 4.68 meses, lo que sugiere que la mayoría de los contratos son de 1, 3 o 6 meses. El valor máximo de 12 meses indica membresías anuales.

Avg_additional_charges_total: Hay una gran variación: desde casi cero hasta más de 550 dólares gastados en servicios adicionales. La media es de 146.94, con una desviación estándar bastante alta.

Avg_class_frequency_total vs. current_month: La frecuencia promedio de asistencia semanal es de alrededor de 1.87 veces por semana históricamente, y 1.76 veces en el último mes. Esto sugiere cierta estabilidad, aunque podría analizarse si una disminución reciente anticipa la cancelación.

Churn (cancelación): La media de 0.2652 indica que aproximadamente el 26.5% de los clientes cancelaron durante el período observado. Esto será útil como referencia base para los modelos predictivos.

In [ ]:
# Promedios por grupo: clientes que cancelaron vs. los que no
df.groupby('Churn').mean()


Near_Location: El 87% de quienes no cancelaron vivían o trabajaban cerca del gimnasio, mientras que solo el 76% de los que cancelaron estaban cerca. Esto sugiere que la proximidad al gimnasio puede influir positivamente en la retención.

Contract_period: Los clientes activos tenían contratos promedio de casi 6 meses, mientras que los que se fueron tenían contratos mucho más cortos (promedio de 1.7 meses). Esto indica que los contratos más largos ayudan a retener clientes.

Group_visits: Participar en clases grupales también parece estar correlacionado con una mayor permanencia. El promedio es 0.46 entre quienes se quedan, contra 0.26 entre quienes se van.

Avg_additional_charges_total: Quienes no cancelaron gastaron más en servicios adicionales (~158 USD vs 115 USD). Esto puede reflejar una mayor implicación o satisfacción con el gimnasio.

Lifetime: El tiempo como cliente es mucho mayor en quienes siguen activos (4.7 meses) en comparación con los que cancelaron (0.99 meses).

Frecuencia de asistencia: Tanto la frecuencia total como la del último mes son significativamente más altas en los clientes que permanecen (más de 2 veces por semana) frente a quienes cancelaron (alrededor de 1 vez por semana).

Estas diferencias sugieren que la cercanía, el tipo de contrato, la participación en actividades grupales, y la frecuencia de uso son variables clave a considerar al predecir la cancelación de clientes.

Vamos a trazar histogramas y gráficos de distribución para observar cómo se comportan las variables más importantes (como edad, tiempo de vida como cliente, frecuencia de asistencia, etc.) en los dos grupos: los que cancelaron (Churn = 1) y los que se quedaron (Churn = 0).

Estas visualizaciones nos permitirán detectar patrones más claros y comparativos entre ambos perfiles de clientes.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ajuste del estilo general
sns.set(style="whitegrid")

# Lista de columnas numéricas que queremos visualizar
columns_to_plot = ['Age', 'Lifetime', 'Avg_class_frequency_total', 'Avg_class_frequency_current_month', 'Avg_additional_charges_total']

# Creamos una figura con subplots
plt.figure(figsize=(18, 15))

# Graficamos cada variable
for i, col in enumerate(columns_to_plot, 1):
    plt.subplot(3, 2, i)
    sns.histplot(data=df, x=col, hue='Churn', kde=True, bins=30, palette='Set2')
    plt.title(f'Distribución de {col} por Cancelación')

plt.tight_layout()
plt.show()


🔸 Edad (Age) Los clientes que cancelaron tienden a concentrarse en un rango más joven, alrededor de los 24-28 años.

En cambio, los clientes que se quedan tienen una distribución más amplia, concentrada en los 28-33 años.

Esto sugiere que los usuarios ligeramente mayores podrían tener mayor compromiso o estabilidad.

🔸 Tiempo como cliente (Lifetime) La mayoría de quienes cancelaron estuvieron menos de 2 meses en el gimnasio.

Los que no cancelaron tienen una distribución más extendida, con muchos clientes de larga duración.

Esto refuerza la idea de que los primeros meses son críticos para retener al cliente.

🔸 Frecuencia promedio total (Avg_class_frequency_total) La frecuencia de visitas semanales es más alta en quienes se quedan (pico cerca de 2 a 3 veces por semana).

Quienes cancelan suelen asistir con menos frecuencia, especialmente por debajo de 1.5 veces por semana.

Esto indica que una mayor participación está fuertemente ligada a la permanencia.

🔸 Frecuencia en el último mes (Avg_class_frequency_current_month) Se repite el patrón anterior: los que no cancelan tienen frecuencias más altas (2–3 veces por semana).

Los que cancelan presentan una caída notable, muchos con menos de 1 visita semanal en el último mes.

Esto podría ser un síntoma de desconexión progresiva del cliente.

🔸 Gastos adicionales (Avg_additional_charges_total) Las personas que gastan más en servicios adicionales tienden a permanecer.

Los clientes que cancelan tienen un patrón más plano y gastos menores.

Esto puede interpretarse como que un mayor involucramiento económico está ligado a una mayor retención.

🔗 Matriz de correlación

Para identificar relaciones entre las variables numéricas, construiremos una matriz de correlación utilizando el coeficiente de Pearson. Este coeficiente varía entre -1 y 1:

1 indica una correlación positiva perfecta.

-1 indica una correlación negativa perfecta.

0 indica ausencia de correlación.

Nos interesa especialmente identificar qué variables tienen mayor correlación con Churn, ya que eso nos dará pistas sobre los factores más relevantes para la cancelación.

# Calculamos la matriz de correlación
corr_matrix = df.corr()

# Visualizamos con un mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Matriz de correlación')
plt.show()


🔹 Variables con mayor correlación negativa con Churn: Contract_period (-0.39): Cuanto más largo es el contrato, menor es la probabilidad de cancelar. Esto confirma que los contratos extensos ayudan a la retención.

Month_to_end_contract (-0.36): Las personas con más tiempo restante en su contrato también tienden a no cancelar.

Avg_class_frequency_current_month (-0.41) y Avg_class_frequency_total (-0.25): La frecuencia de asistencia, tanto histórica como reciente, tiene una relación clara con la permanencia del cliente.

Lifetime (-0.26): A más meses como cliente, menos probabilidad de cancelar.

Group_visits (-0.20): Participar en clases grupales también tiene una correlación inversa moderada con la cancelación.

🔹 Variables con correlación casi nula con Churn: Gender, Phone, Promo_friends, y Partner tienen correlaciones cercanas a 0, lo que indica que no influyen de forma significativa en la cancelación.

Age tiene una correlación muy baja (-0.11), lo que sugiere que no es un factor determinante, aunque combinado con otras variables podría tener cierto impacto.

En resumen, los mejores predictores de cancelación parecen estar relacionados con la duración del contrato, la frecuencia de asistencia, el tiempo como cliente y el grado de involucramiento (visitas grupales, gastos adicionales).

🧪 Paso 3: Construcción del modelo de predicción
🎯 Objetivo Vamos a entrenar dos modelos de clasificación binaria para predecir si un cliente cancelará su membresía en el próximo mes (Churn = 1) o no (Churn = 0). Los modelos que probaremos son:

Regresión logística

Bosque aleatorio (Random Forest)

Antes de entrenar los modelos, debemos preparar nuestros datos:

Definir la variable objetivo y.

Seleccionar las características predictoras X.

Dividir los datos en entrenamiento y validación.

Escalar las variables numéricas para mejorar el rendimiento del modelo de regresión logística.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Variable objetivo
y = df['Churn']

# 2. Variables predictoras (eliminamos 'Churn')
X = df.drop(columns='Churn')

# 3. Dividimos en entrenamiento y validación (80% - 20%)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 4. Estandarización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

Vamos a entrenar los dos modelos con los datos estandarizados y comparar sus resultados. Utilizaremos las siguientes métricas:

Exactitud (Accuracy): proporción de predicciones correctas.

Precisión (Precision): proporción de verdaderos positivos sobre todos los positivos predichos.

Recall: proporción de verdaderos positivos sobre todos los positivos reales.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# 1. Regresión logística
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train_scaled, y_train)
log_preds = log_model.predict(X_valid_scaled)

# 2. Bosque aleatorio
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)  # Notamos que el bosque aleatorio puede trabajar sin escalar
rf_preds = rf_model.predict(X_valid)

# 3. Evaluación
def evaluar_modelo(nombre, y_true, y_pred):
    print(f'📌 {nombre}')
    print(f'Accuracy: {accuracy_score(y_true, y_pred):.3f}')
    print(f'Precision: {precision_score(y_true, y_pred):.3f}')
    print(f'Recall: {recall_score(y_true, y_pred):.3f}')
    print('---')

evaluar_modelo("Regresión Logística", y_valid, log_preds)
evaluar_modelo("Bosque Aleatorio", y_valid, rf_preds)

Ambos modelos muestran alta precisión y recall, lo cual indica que son buenos predictores para identificar clientes que se darán de baja.

El modelo de bosque aleatorio tiene un desempeño ligeramente superior en todas las métricas:

Detecta mejor a los clientes que van a cancelar (mayor recall).

Hace predicciones más confiables (mayor precisión).

Tiene la mayor exactitud global.

Conclusión: el bosque aleatorio es el modelo más recomendado para este caso. Además, tiene la ventaja de manejar bien relaciones no lineales y variables no escaladas.

🧹 Preparación:
Eliminamos la variable objetivo y estandarizamos los datos Primero preparamos los datos para el clustering:

Eliminamos la columna Churn.

Estandarizamos los datos para que todas las variables tengan igual peso.

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. Eliminamos la columna de cancelación
df_cluster = df.drop(columns='Churn')

# 2. Estandarizamos todas las variables
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

# Generamos la matriz de enlaces jerárquicos
linked = linkage(df_scaled, method='ward')  # método 'ward' minimiza la varianza intra-cluster

# Trazamos el dendrograma
plt.figure(figsize=(15, 7))
dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=False)
plt.title('Dendrograma de Clientes')
plt.xlabel('Índice de Cliente')
plt.ylabel('Distancia')
plt.show()

Podemos observar una separación natural en 5 grandes ramas principales, lo que sugiere que usar 5 clústeres es una elección razonable.

Ahora pasaremos a aplicar el algoritmo K-Means para clasificar a los clientes en estos 5 grupos.

🎯 Clustering con K-Means (n = 5)

Utilizaremos el algoritmo de K-Means, que busca agrupar los puntos (clientes) en función de la distancia a los centros de los clústeres. Al usar 5 grupos, podremos identificar distintos perfiles de clientes con comportamientos similares.

In [ ]:
from sklearn.cluster import KMeans

# Entrenamos el modelo con 5 clústeres
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(df_scaled)

# Agregamos la columna 'cluster' al DataFrame original
df['cluster'] = clusters

# Mostramos los primeros registros con su clúster asignado
df[['cluster'] + df.columns[:-1].tolist()].head()

🧠 Análisis de los perfiles por clúster

Vamos a calcular las medias por clúster para entender las características promedio de cada grupo. Esto nos permitirá descubrir qué distingue a cada tipo de cliente.

In [ ]:
# Promedios de características por clúster
df.groupby('cluster').mean().round(2)

🧩 Perfil y análisis de clústeres A partir del agrupamiento de clientes, se identificaron 5 clústeres con características distintas. A continuación se describen los perfiles promedio de cada grupo y su propensión a cancelar (columna Churn):

🔹 Clúster 0 — Compromiso moderado Alta proporción vive cerca (0.95) y tiene pareja asociada (0.83).

Contratos cortos (3 meses) y duración de membresía corta (3.8 meses).

Asisten regularmente, pero su frecuencia ha bajado (3.77 → 1.67).

Tasa de cancelación: 25%

🔹 Clúster 1 — Frecuentes y estables Poco asociados con empresas y menos promociones.

Alta frecuencia actual (4.78) y contratos más cortos (~2.6 meses).

Gastan más en servicios extra (~160).

Tasa de cancelación más baja: 9%

🔹 Clúster 2 — Contratos largos y fieles Altísima duración de contrato (~11.85 meses).

Tienen buena frecuencia histórica (4.68) y mucha antigüedad (~10.8 meses).

Perfil asociado a compañías (~0.74).

Tasa de cancelación más baja: 2%

🔹 Clúster 3 — Muy propensos a cancelar Baja frecuencia de asistencia (histórica y actual).

No están cerca, ni asociados ni traídos por amigos.

Baja permanencia (1.8 meses).

Tasa de cancelación más alta: 57%

🔹 Clúster 4 — Involucrados pero en riesgo Buena frecuencia histórica (3.94), pero bajan su asistencia actual.

Tienen contratos de duración media (~4.7 meses).

Moderada tasa de cancelación.

Tasa de cancelación: 27%

📊 Visualización de características por clúster Trazaremos histogramas agrupados por clúster para variables clave como:

Lifetime

Contract_period

Avg_class_frequency_total

Avg_class_frequency_current_month

Avg_additional_charges_total

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Variables a graficar
features = [
    'Lifetime',
    'Contract_period',
    'Avg_class_frequency_total',
    'Avg_class_frequency_current_month',
    'Avg_additional_charges_total'
]

# Tamaño de figura
plt.figure(figsize=(18, 15))

# Graficar cada variable
for i, col in enumerate(features, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(data=df, x='cluster', y=col, palette='Set2')
    plt.title(f'{col} por Clúster')
    plt.xlabel('Clúster')
    plt.ylabel(col)

plt.tight_layout()
plt.show()


📊 Análisis visual de características por clúster Estas gráficas de caja permiten comparar visualmente los perfiles de cada clúster según características clave:

🔸 Lifetime (tiempo como cliente) El clúster 2 tiene una mayor mediana y más clientes con larga permanencia.

El clúster 3 presenta los tiempos más bajos, lo que coincide con su alta tasa de cancelación.

🔸 Contract_period (duración del contrato) El clúster 2 tiene contratos más largos (muchos de 12 meses).

El clúster 3 tiene contratos muy cortos, lo que puede facilitar la cancelación.

🔸 Avg_class_frequency_total y current_month (frecuencia de asistencia) El clúster 1 destaca por tener la mayor frecuencia tanto histórica como reciente, lo cual coincide con su baja tasa de cancelación.

El clúster 3 tiene frecuencias muy bajas, especialmente en el mes actual, lo que refuerza su riesgo de abandono.

🔸 Avg_additional_charges_total (gastos en servicios adicionales) El clúster 2 y el clúster 1 son los que más gastan, lo que sugiere mayor implicación con el gimnasio.

El clúster 3, nuevamente, destaca por sus bajos niveles.

# Conclusiones y recomendaciones

📌 Conclusiones generales La cancelación afecta aproximadamente al 26% de los clientes, lo cual representa una tasa significativa de pérdida.

Los factores más asociados a la retención de clientes son:

Duración del contrato: contratos largos (6-12 meses) están vinculados a menores tasas de cancelación.

Frecuencia de visitas: tanto histórica como reciente. Una baja frecuencia mensual suele preceder la cancelación.

Participación en clases grupales y gastos adicionales también reflejan mayor compromiso y menor riesgo de abandono.

Clientes nuevos (poca antigüedad) tienen un riesgo alto de cancelar.

👥 Segmentos de clientes detectados (clústeres) Clúster 1 y 2: clientes más fieles, con alta participación, buena frecuencia y contratos más largos. Son un ejemplo a seguir.

Clúster 3: el más crítico, con alta tasa de cancelación, baja frecuencia de visitas, poca antigüedad y contratos cortos.

Clúster 0 y 4: perfiles intermedios, con riesgo moderado que podría reducirse con medidas adecuadas.

# 💡 Recomendaciones para mejorar la retención
Fortalecer las primeras semanas:

Implementar un programa de bienvenida y seguimiento durante el primer mes.

Incentivar la participación en clases grupales desde el inicio.

Promover contratos más largos con beneficios:

Descuentos o beneficios exclusivos para quienes tomen membresías de 6 o 12 meses.

Detectar abandono temprano:

Automatizar alertas cuando la frecuencia mensual baje bruscamente.

Contactar a estos clientes con promociones o motivación personalizada.

Fomentar el uso de servicios adicionales:

Bonificaciones cruzadas (por ejemplo, un masaje gratis al tercer mes).

Programas de lealtad para quienes usan cafetería, tienda o servicios complementarios.

# 🏁 Conclusión final del proyecto

En este proyecto realizamos un análisis integral de los datos de clientes del gimnasio Model Fitness con el objetivo de comprender y reducir la cancelación de membresías. A través del análisis exploratorio, la construcción de modelos predictivos y el agrupamiento de usuarios, obtuvimos hallazgos clave sobre los factores que influyen en la retención.

Se identificó que la frecuencia de visitas, la duración del contrato, la participación en clases grupales y el nivel de gasto en servicios adicionales son variables cruciales en la permanencia del cliente. Además, se identificaron distintos perfiles de usuario mediante clustering, lo que permitió distinguir grupos de alto y bajo riesgo de cancelación.

Gracias a estos análisis, se proponen estrategias claras y aplicables para mejorar la retención, entre ellas: fortalecer el onboarding de nuevos clientes, fomentar contratos a largo plazo, detectar signos tempranos de abandono y promover un mayor uso de servicios complementarios.

Este estudio sienta las bases para una estrategia de marketing y atención más enfocada, personalizada y basada en datos, con el fin de maximizar la lealtad del cliente y la sostenibilidad del negocio.